# Preparing proper inputs for the GMS-borehole module out of OSWCR's database(Open Data)

In [1]:
# %pylab inline
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('WCRs_table_xyz_1.csv')
df

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,OBJECTID,WCR_,X-DMS,Y_DMS,Zval_DEM,X_NAD,Y_NAD,INTERVALST,INTERVALEN,interval_start_depth,interval_end_depth,Soil_ID,DESCRIPTION
0,1,WCR2019-011359,-119.317107,35.054617,693.011919,835927.0813,3885305.457,0,35,693.011919,658.011919,2,sand
1,2,WCR2019-011359,-119.317107,35.054617,693.011919,835927.0813,3885305.457,35,45,658.011919,648.011919,1,peak gravel
2,3,WCR2019-011359,-119.317107,35.054617,693.011919,835927.0813,3885305.457,45,95,648.011919,598.011919,9,sandy clay
3,4,WCR2019-011359,-119.317107,35.054617,693.011919,835927.0813,3885305.457,95,110,598.011919,583.011919,2,sand
4,5,WCR2019-011359,-119.317107,35.054617,693.011919,835927.0813,3885305.457,110,135,583.011919,558.011919,4,clay
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124542,124543,WCR2019-009270,-122.040190,40.505870,1391.512798,581323.0673,4484348.182,74,76,1317.512798,1315.512798,22,broken grey basalt
124543,124544,WCR2019-009270,-122.040190,40.505870,1391.512798,581323.0673,4484348.182,76,89,1315.512798,1302.512798,22,grey basalt
124544,124545,WCR2019-009270,-122.040190,40.505870,1391.512798,581323.0673,4484348.182,89,123,1302.512798,1268.512798,22,red and grey volcanic rock. 50% loss of circ @...
124545,124546,WCR2019-009270,-122.040190,40.505870,1391.512798,581323.0673,4484348.182,123,158,1268.512798,1233.512798,22,grey and red volcanic rock


# Adding empty rows after each well report

In [3]:
# adding empty rows after each well report
l = df[df['INTERVALST']==0].index.tolist()
for c, i in enumerate(l):
    dfs = np.split(df, [i+1+c])
    df = pd.concat([dfs[0], pd.DataFrame([[np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]], \
        columns=df.columns), dfs[1]], ignore_index=True)

# Replacing blank rows with the next row data(backward fill)

# Comparing 2 consecutive rows and if the differnce is zero replace with the next column value

In [4]:
# replacing blank rows with the next row data(backward fill)
data = df.bfill().bfill()
data.index[0]
for idx in range(0, 124545,1):
    temp= idx
    temp2= idx+1
    if data.loc[temp,"INTERVALST"]-data.loc[temp2,"INTERVALST"] == 0:
        data.loc[temp,"INTERVALST"]=data.loc[temp2,"INTERVALEN"]  
        data.loc[temp,"interval_start_depth"]=data.loc[temp2,"interval_end_depth"] 


# Adding Soil Horizons(geologic contacts, lithology change) from bottom top for each well-log

In [5]:
minrow = 0
for idx in range(0,len(data.index)-1):
    if data.loc[idx, "Y_NAD"] != data.loc[idx+1,"Y_NAD"] or idx == len(data.index)-2:
        maxrow = idx
        diff = maxrow-minrow
        for irow in range(diff+1):
            data.loc[minrow+irow,"horizons"] = diff-irow
        rev = data.iloc[minrow:maxrow+1,:]
        if minrow == 0:
            revpd = rev[::-1].reset_index()
        else:
            revpd = pd.concat([revpd,rev[::-1].reset_index()],ignore_index=True)
        minrow = maxrow+1

In [8]:
results = revpd.loc[:,["WCR_","Y_NAD","X_NAD","interval_start_depth","horizons","INTERVALST","Soil_ID","DESCRIPTION"]]
results = results.rename(columns={"Y_NAD": "Y", "X_NAD": "X","interval_start_depth":"Z"})
results.to_csv("df2.csv")